In [1]:
import numpy as np
import pandas as pd
import requests
import io

In [ ]:
url="http://forge.info.univ-angers.fr/~gh/Shspdb/output/fasta_1604232766.txt"
s=requests.get(url)
text=[i.split('\n') for i in s.text.split('\n\n')]
text=[''.join(i[1:])[:-1] for i in text]
protein=pd.DataFrame(text)

protein.to_csv('protein.csv',index=False)

In [2]:
protein =pd.read_csv('protein.csv')

In [3]:
protein.head()

,0
0,MSIVRRSNVFDPFADLWADPFDTFRSIVPAISGGGSETAAFANARM...
1,TVQTSKNPQVDIAEDNAFFPSEYSLSQYTSPVSDLDGVDYPKPYRG...
2,MGHHHHHHSHMRKIDLCLSSEGSEVILATSSDEKHPPENIIDGNPE...
3,MQLSSGVSEIRHTADRWRVSLDVNHFAPDELTVKTKDGVVEITGKH...
4,GAMAQVPTDPGYFSVLLDVKHFSPEEISVKVVGDHVEVHARHEERP...


In [6]:
A="abcdef"
B="ab"

In [242]:
A=protein.iloc[0].values[0]
B=protein.iloc[1].values[0]

# Dynamic Programming

Space Complexity O(mn)

In [4]:
%time
def ed_dp(A,B):
    """
    Minimum Edit Distance ( Classical Algorithm/ Dynamic Programming )
    A ----> Initial State
    B ----> Goal State   
    """
    m=len(A)
    n=len(B)
    
    c=np.zeros(shape=(m+1,n+1))
    d=[[None for i in range(n+1)] for i in range(m+1)]
    
    for i in range(m+1):
        c[i,0]=i
        if i==0:
            d[0][0]='0'
        else:
            d[i][0]=A[i-1]
    for i in range(n+1):
        c[0,i]=i
        if i==0:
            d[0][0]='0'
        else:
            d[0][i]=B[i-1]
    
    for i in range(1,m+1):
        for j in range(1,n+1):
            k=min([c[i-1,j-1],c[i-1,j],c[i,j-1]])
            if k==c[i-1][j-1]:
                d[i][j]="`"
            elif k==c[i-1][j]:
                d[i][j]="^"
            else:
                d[i][j]="<"
            
            
            if A[i-1]==B[j-1]:
                c[i][j]=k
            else:
                c[i][j]=k+1
                if k==c[i-1][j-1]:
                    d[i][j]="s"
    print(f"No of Changes {c[-1][-1]}")
    return c,d

def get_alignment(A,B,d):
    m=len(d)
    n=len(d[0])-1
    i=m-1
    while i>0:
        if d[i][n]=='s':
            print(f'Subsititute {A[i-1],i-1} with {B[n-1]}')
            n-=1
            i-=1
        elif d[i][n]=='^':
            print(f'Delete {A[i-1],i-1}')
            i-=1
        elif d[i][n]=="<":
            print(f'Add {B[n-1]} between {A[i-1],i-1} and {A[i],i}')
            n-=1
        elif d[i][n]=='`':
            n-=1
            i-=1

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11.2 µs


In [7]:
A

'abcdef'

In [8]:
ed_dp('Sunday','Saturday')

No of Changes 3.0


(array([[0., 1., 2., 3., 4., 5., 6., 7., 8.],
        [1., 0., 1., 2., 3., 4., 5., 6., 7.],
        [2., 1., 1., 2., 2., 3., 4., 5., 6.],
        [3., 2., 2., 2., 3., 3., 4., 5., 6.],
        [4., 3., 3., 3., 3., 4., 3., 4., 5.],
        [5., 4., 3., 4., 4., 4., 4., 3., 4.],
        [6., 5., 4., 4., 5., 5., 5., 4., 3.]]),
 [['0', 'S', 'a', 't', 'u', 'r', 'd', 'a', 'y'],
  ['S', '`', '<', '<', '<', '<', '<', '<', '<'],
  ['u', '^', 's', 's', '`', '<', '<', '<', '<'],
  ['n', '^', 's', 's', 's', 's', 's', 's', 's'],
  ['d', '^', 's', 's', 's', 's', '`', '<', '<'],
  ['a', '^', '`', 's', 's', 's', '^', '`', '<'],
  ['y', '^', '^', 's', 's', 's', 's', '^', '`']])

In [9]:
get_alignment('Sunday','Saturday',ed_dp('Sunday','Saturday')[1])

No of Changes 3.0
Subsititute ('n', 2) with r
Add t between ('S', 0) and ('u', 1)
Add a between ('S', 0) and ('u', 1)


In [247]:
%time
get_alignment(A,B,ed_dp(A,B)[1])

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 13.6 µs
No of Changes 208.0
Subsititute ('S', 149) with A
Subsititute ('I', 148) with Y
Subsititute ('Q', 147) with A
Delete ('I', 146)
Subsititute ('K', 144) with L
Subsititute ('V', 143) with M
Subsititute ('P', 141) with Q
Subsititute ('K', 140) with A
Add A between ('K', 139) and ('K', 140)
Add L between ('K', 139) and ('K', 140)
Subsititute ('V', 138) with G
Subsititute ('E', 137) with L
Add A between ('A', 136) and ('E', 137)
Add N between ('A', 136) and ('E', 137)
Add A between ('A', 136) and ('E', 137)
Subsititute ('K', 135) with F
Subsititute ('V', 133) with S
Subsititute ('T', 132) with D
Subsititute ('V', 131) with G
Add L between ('L', 129) and ('T', 130)
Add K between ('V', 128) and ('L', 129)
Add R between ('V', 128) and ('L', 129)
Add D between ('V', 128) and ('L', 129)
Add K between ('V', 128) and ('L', 129)
Add H between ('V', 128) and ('L', 129)
Add R between ('G', 127) and ('V', 128)
Add T between ('N', 126) and

In [231]:
get_alignment('Sunday','Saturday',ed_dp('Sunday','Saturday')[1])

No of Changes 3.0
Subsititute ('n', 2) with r
Add t between ('S', 0) and ('u', 1)
Add a between ('S', 0) and ('u', 1)


# Levenshtein Distance

In [ ]:
def ed_recursive(A,B,i,j):
    """Python Implementation based on Levenshtein Algorithm"""